Wills Kookogey PyTorch Learning Enviromnment

## Setup and config

In [1]:
import fennec_ml as fn
import json
import os
from pathlib import Path
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms  as transforms
import matplotlib.pyplot as plt
import get_FID_labels
%matplotlib inline

# hyper parameters
input_size = 12
hidden_size = 128
num_classes = 9
num_epochs = 25
batch_size = 100
learning_rate = 0.01

sequence_length = 10
num_layers = 1

In [2]:
# folder config
cwd = os.getcwd()
main_dir = os.path.dirname(cwd)
data_dir = os.path.join(main_dir, "DATA")
os.makedirs(data_dir, exist_ok=True)
excel_dir = os.path.join(data_dir, "RAW_DATA/FID_1")
os.makedirs(excel_dir, exist_ok=True)
csv_dir = os.path.join(data_dir, "PROCESSED_DATA/FID_1")
os.makedirs(csv_dir, exist_ok=True)

json_path = os.path.join(main_dir, "vars_of_interest.json")

# convert excel data to csv
print("Using folder_cleaner() to convert from raw excel files to useful .csv's.\nfolder_cleaner() output:")
fn.folder_cleaner(excel_dir, csv_dir, overwrite= True)

# convert csv to numpy arrays
scaled_data = fn.standardize(csv_dir)
# scaled_data = fn.normalize(csv_dir)
labels = get_FID_labels.get_FID_labels(csv_dir)

# segment and split data
print("\nUsing segment_and_split() to cut data into training/validtion/testing sets\nsegment_and_split() output:")
# using default 70%/15%/15% train/val/test split
dataset_dict = fn.segment_and_split(scaled_data, labels, timesteps=sequence_length)

Using folder_cleaner() to convert from raw excel files to useful .csv's.
folder_cleaner() output:
105G_R.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 105G_R.csv
102G_NONE.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 102G_NONE.csv
107G_LR.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 107G_LR.csv
104G_L.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 104G_L.csv
103G_LR.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 103G_LR.csv
109G_L.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA/FID_1 as 109G_L.csv
106G_NONE.xlsx process

## Check Data Shape

In [3]:
print(dataset_dict["Training_Set"]["sets"].shape)
print(dataset_dict["Training_Set"]["labels"][0])

(1909, 10, 12)
LR


## Convert data to tensors

In [4]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, TensorDataset
import numpy as np

encoder = LabelEncoder()

training_labels = dataset_dict["Training_Set"]["labels"]
encoded_labels_train = encoder.fit_transform(training_labels)

validation_labels = dataset_dict["Validation_Set"]["labels"]
encoded_labels_val = encoder.fit_transform(validation_labels)

testing_labels = dataset_dict["Testing_Set"]["labels"]
encoded_labels_test = encoder.fit_transform(testing_labels)

print(encoded_labels_train)
print(encoder.classes_)

input_train = torch.from_numpy(dataset_dict["Training_Set"]["sets"]).type(torch.float)
output_train = torch.from_numpy(encoded_labels_train).type(torch.int)
input_val = torch.from_numpy(dataset_dict["Validation_Set"]["sets"]).type(torch.float)
output_val = torch.from_numpy(encoded_labels_val).type(torch.int)
input_test = torch.from_numpy(dataset_dict["Testing_Set"]["sets"]).type(torch.float)
output_test = torch.from_numpy(encoded_labels_test).type(torch.int)

# stack inputs and outputs
train_data = TensorDataset(input_train, output_train)
val_data = TensorDataset(input_val, output_val)
test_data = TensorDataset(input_test, output_test)

# data loader
class_counts = np.bincount(output_train)
weights = 1. / class_counts
sample_weights = [weights[label] for label in output_train]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_data, batch_size=batch_size, sampler=sampler)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

[1 1 3 ... 2 3 1]
['L' 'LR' 'NONE' 'R']


# Playground

In [5]:
from pathlib import Path
import torch
import torch.nn as nn
from torch.optim import Adam
import torchvision
import torchvision.transforms  as transforms
import matplotlib.pyplot as plt
%matplotlib inline

# Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# Create model save path
MODEL_NAME = "fid_model_1.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Device config
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device found.")
else:
    print("MPS device not found. Using CPU.")
    device = torch.device("cpu") # Fallback to CPU if MPS is not available

# Ben Keller 1D Model
class ResidualGRU(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, num_classes=5, dropout=0.3):
        super(ResidualGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers,
                          batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size // 2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        # GRU output
        out, _ = self.gru(x)   # out: [batch, seq, hidden]
        
        # Take only the last timestep
        out = out[:, -1, :]    # [batch, hidden]

        # Residual connection (skip last layer’s features through dense layers)
        residual = out.clone()

        # Fully connected layers with dropout + residual
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

model = ResidualGRU(input_size=12, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
best_val_loss = float('inf')

MPS device found.


In [6]:
# Train Loop

for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0
    for input, output in train_loader:
        input, output = input.to(device), output.to(device)
        optimizer.zero_grad()
        model_outputs = model(input)
        loss = criterion(model_outputs,output)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * input.size(0)
    epoch_loss = running_loss / len(train_data)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs,y_batch)
            val_loss += loss.item() * X_batch.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y_batch).sum().item()
    val_loss /= len(val_data)
    val_acc = correct / len(val_data)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"✅ Saved best model at epoch {epoch+1}")

print("Training complete!")

Epoch 1/25, Train Loss: 1.8696, Val Loss: 1.2329, Val Acc: 0.5990
✅ Saved best model at epoch 1
Epoch 2/25, Train Loss: 1.1389, Val Loss: 0.8940, Val Acc: 0.6406
✅ Saved best model at epoch 2
Epoch 3/25, Train Loss: 0.9625, Val Loss: 0.8474, Val Acc: 0.6650
✅ Saved best model at epoch 3
Epoch 4/25, Train Loss: 0.8597, Val Loss: 0.7820, Val Acc: 0.6699
✅ Saved best model at epoch 4
Epoch 5/25, Train Loss: 0.8103, Val Loss: 0.6896, Val Acc: 0.7042
✅ Saved best model at epoch 5
Epoch 6/25, Train Loss: 0.7453, Val Loss: 0.6850, Val Acc: 0.6993
✅ Saved best model at epoch 6
Epoch 7/25, Train Loss: 0.6862, Val Loss: 0.5490, Val Acc: 0.7873
✅ Saved best model at epoch 7
Epoch 8/25, Train Loss: 0.5879, Val Loss: 0.4941, Val Acc: 0.7995
✅ Saved best model at epoch 8
Epoch 9/25, Train Loss: 0.4670, Val Loss: 0.3880, Val Acc: 0.8435
✅ Saved best model at epoch 9
Epoch 10/25, Train Loss: 0.4129, Val Loss: 0.3528, Val Acc: 0.8680
✅ Saved best model at epoch 10
Epoch 11/25, Train Loss: 0.3728, Val L

Output

In [7]:
import matplotlib.pyplot as plt
from scipy import stats

correct_guesses = 0
total_guesses = 0
correctness = 0

model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.to(device)
model.eval()

for inputs, labels in test_loader:   # each iteration = 1 file dataset tensor
    inputs = inputs.to(device)
    
    with torch.no_grad():
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()

    # Average prediction for the file
    mean_preds = probs.mean(axis=0)
    mode_true = stats.mode(labels.numpy())
    predicted_class = encoder.classes_[np.argmax(mean_preds)]
    true_classes = encoder.classes_[labels]
    true_class_avg = encoder.classes_[mode_true.mode]

    print(f"File: {true_class_avg}, Predicted: {predicted_class}")

    # tally correct guesses
    if (predicted_class == true_class_avg):
        correct_guesses += 1
    
    total_guesses += 1

# calculate correctness
correctness = correct_guesses / total_guesses
print(f"Correctness Percentage: {correctness}")

File: NONE, Predicted: NONE
File: LR, Predicted: LR
File: NONE, Predicted: NONE
File: LR, Predicted: LR
File: LR, Predicted: LR
Correctness Percentage: 1.0
